In [1]:
#import warnings
#warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [2]:
weather_df = pd.read_csv('fiveyearsweatherdata.csv', header = 0) 
weather_df.head()

,Address,Date time,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,...,Visibility,Cloud Cover,Sea Level Pressure,Weather Type,Latitude,Longitude,Resolved Address,Name,Info,Conditions
0,"Kolkata,West Bengal,India",01/01/2017,14.0,25.3,19.9,16.5,82.47,NaN,5.8,NaN,...,1.2,4.9,1015.9,"Mist, Fog, Smoke Or Haze",22.5705,88.3713,"Kolkata, WB, India","Kolkata, WB, India",NaN,Clear
1,"Kolkata,West Bengal,India",01/02/2017,16.0,26.0,20.6,16.0,76.76,NaN,8.1,NaN,...,1.4,0.3,1015.6,"Mist, Fog, Smoke Or Haze",22.5705,88.3713,"Kolkata, WB, India","Kolkata, WB, India",NaN,Clear
2,"Kolkata,West Bengal,India",01/03/2017,16.0,27.0,21.1,15.6,73.12,27.3,9.2,NaN,...,1.6,10.7,1014.7,"Mist, Smoke Or Haze",22.5705,88.3713,"Kolkata, WB, India","Kolkata, WB, India",NaN,Clear
3,"Kolkata,West Bengal,India",01/04/2017,16.0,25.0,19.9,14.5,72.57,NaN,8.1,NaN,...,1.8,2.3,1014.6,"Mist, Smoke Or Haze",22.5705,88.3713,"Kolkata, WB, India","Kolkata, WB, India",NaN,Clear
4,"Kolkata,West Bengal,India",01/05/2017,15.0,25.6,19.6,13.7,71.88,NaN,8.1,NaN,...,1.9,0.0,1013.7,"Mist, Smoke Or Haze",22.5705,88.3713,"Kolkata, WB, India","Kolkata, WB, India",NaN,Clear


In [3]:
weather_df.drop(columns = ['Info', 'Address', 'Resolved Address', 'Name', 'Date time', 'Weather Type', 'Conditions'], inplace = True)

In [4]:
weather_df.head()

,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,Wind Direction,Wind Chill,Precipitation,Precipitation Cover,Snow Depth,Visibility,Cloud Cover,Sea Level Pressure,Latitude,Longitude
0,14.0,25.3,19.9,16.5,82.47,NaN,5.8,NaN,78.33,NaN,0.0,0.0,0.0,1.2,4.9,1015.9,22.5705,88.3713
1,16.0,26.0,20.6,16.0,76.76,NaN,8.1,NaN,148.54,NaN,0.0,0.0,0.0,1.4,0.3,1015.6,22.5705,88.3713
2,16.0,27.0,21.1,15.6,73.12,27.3,9.2,NaN,244.92,NaN,0.0,0.0,0.0,1.6,10.7,1014.7,22.5705,88.3713
3,16.0,25.0,19.9,14.5,72.57,NaN,8.1,NaN,244.38,NaN,0.0,0.0,0.0,1.8,2.3,1014.6,22.5705,88.3713
4,15.0,25.6,19.6,13.7,71.88,NaN,8.1,NaN,231.58,NaN,0.0,0.0,0.0,1.9,0.0,1013.7,22.5705,88.3713


In [5]:
weather_df = weather_df.fillna(weather_df.mean())

In [6]:
weather_df.isna().sum()

Minimum Temperature    0
Maximum Temperature    0
Temperature            0
Dew Point              0
Relative Humidity      0
Heat Index             0
Wind Speed             0
Wind Gust              0
Wind Direction         0
Wind Chill             0
Precipitation          0
Precipitation Cover    0
Snow Depth             0
Visibility             0
Cloud Cover            0
Sea Level Pressure     0
Latitude               0
Longitude              0
dtype: int64

In [7]:
weather_df.head()

,Minimum Temperature,Maximum Temperature,Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,Wind Direction,Wind Chill,Precipitation,Precipitation Cover,Snow Depth,Visibility,Cloud Cover,Sea Level Pressure,Latitude,Longitude
0,14.0,25.3,19.9,16.5,82.47,40.563588,5.8,30.265517,78.33,8.730769,0.0,0.0,0.0,1.2,4.9,1015.9,22.5705,88.3713
1,16.0,26.0,20.6,16.0,76.76,40.563588,8.1,30.265517,148.54,8.730769,0.0,0.0,0.0,1.4,0.3,1015.6,22.5705,88.3713
2,16.0,27.0,21.1,15.6,73.12,27.300000,9.2,30.265517,244.92,8.730769,0.0,0.0,0.0,1.6,10.7,1014.7,22.5705,88.3713
3,16.0,25.0,19.9,14.5,72.57,40.563588,8.1,30.265517,244.38,8.730769,0.0,0.0,0.0,1.8,2.3,1014.6,22.5705,88.3713
4,15.0,25.6,19.6,13.7,71.88,40.563588,8.1,30.265517,231.58,8.730769,0.0,0.0,0.0,1.9,0.0,1013.7,22.5705,88.3713


In [8]:
y = weather_df.pop('Temperature')

In [9]:
y = y.to_numpy()
y

array([19.9, 20.6, 21.1, ..., 20. , 20.2, 21.1])

In [10]:
weather_df.head()

,Minimum Temperature,Maximum Temperature,Dew Point,Relative Humidity,Heat Index,Wind Speed,Wind Gust,Wind Direction,Wind Chill,Precipitation,Precipitation Cover,Snow Depth,Visibility,Cloud Cover,Sea Level Pressure,Latitude,Longitude
0,14.0,25.3,16.5,82.47,40.563588,5.8,30.265517,78.33,8.730769,0.0,0.0,0.0,1.2,4.9,1015.9,22.5705,88.3713
1,16.0,26.0,16.0,76.76,40.563588,8.1,30.265517,148.54,8.730769,0.0,0.0,0.0,1.4,0.3,1015.6,22.5705,88.3713
2,16.0,27.0,15.6,73.12,27.300000,9.2,30.265517,244.92,8.730769,0.0,0.0,0.0,1.6,10.7,1014.7,22.5705,88.3713
3,16.0,25.0,14.5,72.57,40.563588,8.1,30.265517,244.38,8.730769,0.0,0.0,0.0,1.8,2.3,1014.6,22.5705,88.3713
4,15.0,25.6,13.7,71.88,40.563588,8.1,30.265517,231.58,8.730769,0.0,0.0,0.0,1.9,0.0,1013.7,22.5705,88.3713


In [11]:
x = weather_df.to_numpy()
x

array([[  14.    ,   25.3   ,   16.5   , ..., 1015.9   ,   22.5705,
          88.3713],
       [  16.    ,   26.    ,   16.    , ..., 1015.6   ,   22.5705,
          88.3713],
       [  16.    ,   27.    ,   15.6   , ..., 1014.7   ,   22.5705,
          88.3713],
       ...,
       [  15.    ,   26.    ,   11.5   , ..., 1017.    ,   22.5705,
          88.3713],
       [  14.9   ,   26.    ,   13.6   , ..., 1017.5   ,   22.5705,
          88.3713],
       [  15.1   ,   27.    ,   16.    , ..., 1016.4   ,   22.5705,
          88.3713]])

In [12]:
x.shape, y.shape

((4017, 17), (4017,))

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle = True, test_size = 0.1)

In [14]:
x_train.shape, x_test.shape

((3615, 17), (402, 17))

In [15]:
model = LinearRegression()
model.fit(x_train, y_train)

LinearRegression()

In [16]:
y_pred = model.predict(x_test)

In [17]:
mean_absolute_percentage_error(y_pred, y_test)

0.006391587539426368

In [18]:
mean_absolute_error(y_pred, y_test)

0.1655981544094822